# Import and clean downloaded weather data

In [1]:
'''
Author: Brian Mukeswe
Institution: The University of Edinburgh
Date: May 17, 2019

'''
import pandas as pd
import numpy as np

def append_row(data_frame, row_items):
    '''Add a new row to a data frame'''

    item_array = np.array(row_items).reshape(1,len(data_frame.columns))
    
    item_data = pd.DataFrame(item_array,columns=data_frame.columns) 
    
    return pd.concat([data_frame, item_data], axis=0)


def insert_line(line, data, station, index):
    ''' Read the data entries on a given line in the text file
        return: A list containing the entries on the line
    '''
    cols = line.strip().split(" ")
    items = [station]
    
    for item in cols:
        if item!='':
            
            # Clean data entries
            clean_item = item.replace("*","")
            clean_item = clean_item.replace("#","")
            clean_item = clean_item.replace("---","1000") # 1000 is used to indicate a missing value
            clean_item = clean_item.replace("$", "")
            
            items.append(clean_item)
            if len(items)>=8:
                items.append(index)
                index += 1
                break
    
    # DEBUGGING
    if len(items)<8:
        print(station, items)
        
    # make sure the row item matches the dataframe size
    if len(items)==len(data.columns):
        data = append_row(data, items)   
        
    return data, index

In [2]:
def get_names(name_list):
    '''Retrieves and cleans list of station names'''
    names = open(name_list, "r").readlines()
    clean_names = []
    
    for name in names:
        clean_names.append(name.replace("\n", ""))
        
    return clean_names

In [3]:
def add_station(station, data, index):
    '''Adds the data of a specified station to a dataframe'''
    
    lines = open("data\\"+station+".txt").readlines()
    
    for line in range(8, len(lines)):
        data, index = insert_line(lines[line], data, station, index)
        
    return data, index

## Load data from all stations into a dataframe

In [4]:
column_names = ["station_name", "year", "month", "tmax_degC", "tmin_degC", "af_days", "rain_mm", "sun_hours", "INDEX"]
data = pd.DataFrame(columns=column_names)

stations = get_names("stations.txt")
index = 0

for station in stations:
    data, index = add_station(station, data, index)
    print("successfully added:", station)
    
data.shape

successfully added: aberporth
successfully added: armagh
successfully added: ballypatrick
successfully added: bradford
successfully added: braemar
successfully added: camborne
successfully added: cambridge
successfully added: cardiff
successfully added: chivenor
cwmystwyth ['cwmystwyth', 'Site', 'closed']
successfully added: cwmystwyth
successfully added: dunstaffnage
successfully added: durham
successfully added: eastbourne
successfully added: eskdalemuir
successfully added: heathrow
successfully added: hurn
successfully added: lerwick
successfully added: leuchars
lowestoft ['lowestoft', '1945', '3', '11.8', '4.1', '1', '35.8']
successfully added: lowestoft
successfully added: manston
successfully added: nairn
successfully added: newtonrigg
successfully added: oxford
successfully added: paisley
ringway ['ringway', 'Site', 'Closed']
successfully added: ringway
successfully added: rossonwye
successfully added: shawbury
successfully added: sheffield
southampton ['southampton', 'Site', 'C

successfully added: whitby
successfully added: wickairport
successfully added: yeovilton


(37086, 9)

Issues observed:
- no sun_hours data entries for some months in whitby.
- no sun_hours data entry for lowesoft 1945 march.
- cwmystwyth, ringway and southampton sites closed at some point.

about 37k data points in total

Need to clean and add skipped data points to dataframe ...

In [5]:
# Create an updated version of the insert_line and add station functions to 
# include skipped data points from lowesoft and whitby

def insert_line_updated(line, data, station, index):
    ''' Read the data entries on a given line in the text file
        return: A list containing the entries on the line
    '''
    cols = line.strip().split(" ")
    items = [station]
    
    for item in cols:
        if item!='':
            
            # Clean data entries
            clean_item = item.replace("*","")
            clean_item = clean_item.replace("#","")
            clean_item = clean_item.replace("---","1000") # 1000 is used to indicate a missing value
            clean_item = clean_item.replace("$", "")
            
            items.append(clean_item)
            if len(items)>=8:
                items.append(index)
                index += 1
                break
    
    # insert 1000 in missing sun_hour data entries
    if len(items)<8:
        if len(items)==7:
            items.append("1000")
            items.append(index)
            index += 1
            
        else:
            print(station, items)
        
    # make sure the row item matches the dataframe size
    if len(items)==len(data.columns):
        data = append_row(data, items)   
        
    return data, index

def add_station_updated(station, data, index):
    '''Adds the data of a specified station to a dataframe'''
    
    lines = open("data\\"+station+".txt").readlines()
    
    for line in range(8, len(lines)):
        data, index = insert_line_updated(lines[line], data, station, index)
        
    return data, index

column_names = ["station_name", "year", "month", "tmax_degC", "tmin_degC", "af_days", "rain_mm", "sun_hours", "INDEX"]
data = pd.DataFrame(columns=column_names)

stations = get_names("stations.txt")
index = 0

for station in stations:
    data, index = add_station_updated(station, data, index)
    print("successfully added:", station)
    
data.shape

successfully added: aberporth
successfully added: armagh
successfully added: ballypatrick
successfully added: bradford
successfully added: braemar
successfully added: camborne
successfully added: cambridge
successfully added: cardiff
successfully added: chivenor
cwmystwyth ['cwmystwyth', 'Site', 'closed']
successfully added: cwmystwyth
successfully added: dunstaffnage
successfully added: durham
successfully added: eastbourne
successfully added: eskdalemuir
successfully added: heathrow
successfully added: hurn
successfully added: lerwick
successfully added: leuchars
successfully added: lowestoft
successfully added: manston
successfully added: nairn
successfully added: newtonrigg
successfully added: oxford
successfully added: paisley
ringway ['ringway', 'Site', 'Closed']
successfully added: ringway
successfully added: rossonwye
successfully added: shawbury
successfully added: sheffield
southampton ['southampton', 'Site', 'Closed']
successfully added: southampton
successfully added: storn

(37259, 9)

In [6]:
# Set the index for the created data frame
data.set_index("INDEX", inplace=True)
data.head(10)

,station_name,year,month,tmax_degC,tmin_degC,af_days,rain_mm,sun_hours
INDEX,,,,,,,,
0,aberporth,1941,2,1000,1000,1000,69.1,1000
1,aberporth,1941,3,1000,1000,1000,76.2,1000
2,aberporth,1941,4,1000,1000,1000,33.7,1000
3,aberporth,1941,5,1000,1000,1000,51.3,1000
4,aberporth,1941,6,1000,1000,1000,25.7,1000
5,aberporth,1941,7,1000,1000,1000,53.9,1000
6,aberporth,1941,8,1000,1000,1000,91.8,1000
7,aberporth,1941,9,1000,1000,1000,25.5,1000
8,aberporth,1941,10,1000,1000,1000,106.2,1000


In [7]:
data.tail(10)

,station_name,year,month,tmax_degC,tmin_degC,af_days,rain_mm,sun_hours
INDEX,,,,,,,,
37249,yeovilton,2018,7,25.9,13.5,0,13.8,242.3
37250,yeovilton,2018,8,22.7,12.8,0,53.8,138.5
37251,yeovilton,2018,9,19.6,9.4,0,36.4,149.0
37252,yeovilton,2018,10,15.4,6.2,4,51.8,132.8
37253,yeovilton,2018,11,12.1,5.3,4,102.6,73.8
37254,yeovilton,2018,12,11.0,5.3,2,94.2,27.2
37255,yeovilton,2019,1,7.8,1.0,14,33.8,53.2
37256,yeovilton,2019,2,11.8,1.9,10,47.2,111.4
37257,yeovilton,2019,3,12.3,4.8,2,66.0,110.0


## Setup methods to store data in a mongodb database server

In [8]:
from pymongo import MongoClient

In [9]:
def make_lineObj(line, data):
    ''' create a dict object of a specified row
        of a dataframe
    '''
   
    obj = {}
    
    for column in data.columns:
        obj[column] = data[column].iloc[line]
        
    return obj


def storeObj(obj):
    collection = "weather_data"
    
    client = MongoClient(host="local_host", port=27017)
    db = client.pitds_weather_data
    weather_data = db[collection]
    
    weather_data.insert_one(obj)

## set data types before storing in database

In [10]:
# back up data frame in excel file and raw_data variable
raw_data = data.copy(deep=True)
data.to_excel("weather_data.xlsx")

In [11]:
# Check cleaning
# identify offending values that were not succesfully cleaned
sun = data.sun_hours
sun[sun.str.contains("a")]

INDEX
17559    152.0Change
35109        71.6all
Name: sun_hours, dtype: object

In [12]:
# clean letters and numbers mixed within the sun_hours data
sun.loc["17559"] = sun.loc["17559"].replace("Change", "")
sun.loc["35109"] = sun.loc["35109"].replace("all", "")

In [13]:
# Set data types
data.year = data.year.astype(int)
data.month = data.month.astype(int)
data.tmax_degC = data.tmax_degC.astype(float)
data.tmin_degC = data.tmin_degC.astype(float)
data.af_days = data.af_days.astype(int)
data.rain_mm = data.rain_mm.astype(float)
data.sun_hours = data.sun_hours.astype(float)
data.index = data.index.astype(int)

In [21]:
data.sun_hours[data.sun_hours==1000] = None
data.rain_mm[data.rain_mm==1000] = None
data.af_days[data.af_days==1000] = None
data.tmin_degC[data.tmin_degC==1000] = None
data.tmax_degC[data.tmax_degC==1000] = None

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anac

In [22]:
data.describe()

,year,month,tmax_degC,tmin_degC,af_days,rain_mm,sun_hours
count,37259.000000,37259.000000,36338.000000,36364.000000,34944.000000,36394.000000,28786.000000
mean,1968.503986,6.491398,12.722541,5.919173,3.515511,72.784676,118.431227
std,36.546454,3.453688,5.036415,4.059429,5.208213,48.157576,63.166909
min,1853.000000,1.000000,-0.900000,-8.600000,0.000000,0.000000,2.800000
25%,1949.000000,3.000000,8.500000,2.700000,0.000000,39.100000,64.800000
50%,1976.000000,6.000000,12.400000,5.500000,1.000000,62.400000,111.900000
75%,1997.000000,9.000000,16.800000,9.400000,6.000000,94.600000,163.675000
max,2019.000000,12.000000,28.300000,17.000000,31.000000,568.800000,350.300000


In [14]:
# update backed up data instances with formatted datatype
raw_data = data.copy(deep=True)
data.to_excel("weather_data.xlsx")

## store specified station data in a database (using parallel processes)

In [2]:
# Using multiple processes to speed up loading of all data into thedatabase.

def store_lines(batch):
    
    batch_dict = {"b1" : [0, 9200],
                  "b2" : [9200, 18400],
                  "b3" : [18400, 27600],
                  "b4" : [24600, 37259]
                 }
    
    start = batch_dict[batch][0]
    end = batch_dict[batch][1]

    worked = []
    for row in range(start, end):
        storeObj(make_lineObj(row, data))
        worked.append(row)
        
    
        # report every 2000 entries
        if row%2000 == 0:
            print("stored entries:", row)
            
    return worked

from multiprocessing import Pool
import store_lines

if __name__ ==  '__main__':
    
    batches = ("b1", "b2", "b3", "b4")
    num_processors = 2 # specify number of processes to use
    p=Pool(processes = num_processors)
    output = p.map(store_lines.store_lines,batches)

## store specified station data in a database (using single process)

In [15]:
for row in range(9857, len(data)):
    storeObj(make_lineObj(row, data))
    
    # report every 2000 entries
    if row%2000 == 0:
        print("stored entries:", row)

stored entries: 10000
stored entries: 12000
stored entries: 14000
stored entries: 16000
stored entries: 18000
stored entries: 20000
stored entries: 22000
stored entries: 24000
stored entries: 26000
stored entries: 28000
stored entries: 30000
stored entries: 32000
stored entries: 34000
stored entries: 36000
